In [27]:
from salad.datasets import DigitsLoader
import torch
from salad.datasets.da import NoiseLoader
from salad.datasets.transforms import SaltAndPepper
from salad.utils import evaluate

## Generate predictions for digit dataset multi-class training

In [28]:
import glob

def get_data(source):
    loader = NoiseLoader('/tmp/data', source, collate = 'stack', noisemodels=[SaltAndPepper(0.15)],batch_size = 32, shuffle = False, split="test", normalize=False)
    return loader

def run_eval(source, solver, loader):

    fname = list(glob.glob("log/noise-{}-snp/*{}*/*-checkpoint-ep9.pth".format(source, solver)))
    if len(fname) == 0:
        return {}
    
    fname = fname[0]
    
    print(fname)

    acc = {}

    lbl, pred, _ = evaluate.evaluate([fname], loader.datasets[0], domain = 0)
    acc[(source, solver)] = [lbl, pred]
            
    return acc

In [30]:
import itertools

predictions = {}
sources = ['synth']#, 'svhn']
solvers = [
    #"AssociativeSolver",
    #"DANNSolver",
    "DeepCoralSolver",
    #"SelfEnsemblingSolver",
    #"VADASolver"
]

for source in sources:
    loader = get_data(source)
    for solver in solvers:
        predictions.update(run_eval(source, solver, loader))

Extracting /tmp/data/synth_test_32x32.mat?raw=true


  2%|▏         | 7/299 [00:00<00:04, 64.03it/s]

log/noise-synth-snp/20181026-041401_DeepCoralSolver/20181026-041401-checkpoint-ep9.pth


100%|██████████| 299/299 [00:05<00:00, 58.00it/s]


In [31]:
import h5py

with h5py.File("results/noiseadapt_predictions.hdf5") as ds:
    
    for key in predictions:
        
        ds_key = "-".join(key)
        
        grp = ds.require_group(ds_key)
        
        lbls, pred = predictions[key]
        
        grp["prediction"] = pred
        grp["labels"] = lbls

In [21]:
! du -h results/noiseadapt_predictions.hdf5

2.0M	results/noiseadapt_predictions.hdf5
